In [1]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))

In [1]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/P1015.3. Analyses/P1015.3.2. Data Processing/Output'
cur_date = '060625'

library(missForest)
library(readxl)
library(openxlsx)
library(tidyverse)
library(factoextra)

# reading in files
ws_df = data.frame(Sample = 'WS', read_excel("Input/Woodsmoke_Data_053125.xlsx", sheet = 2)) #%>%
    #select(-Sample_Number)
wf_df = data.frame(Sample = 'WF', read_excel("Input/Wildfire_Data_053125.xlsx", sheet = 2)) #%>%
    #select(-Sample_Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [2]:
head(ws_df)
head(wf_df)

,Sample,HAWC_ID,Study,Sample_Number,Number,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,429174,Arif et al. 2017,1,1,PAH,Weight,DTXSID6025648,1-Methylphenanthrene,NA
2,WS,429174,Arif et al. 2017,1,2,PAH,Weight,DTXSID0025654,1-Methylpyrene,NA
3,WS,429174,Arif et al. 2017,1,3,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,NA
4,WS,429174,Arif et al. 2017,1,4,PAH,Weight,DTXSID4063030,2-Methylpyrene,NA
5,WS,429174,Arif et al. 2017,1,5,PAH,Weight,DTXSID50187178,4-Methylpyrene,NA
6,WS,429174,Arif et al. 2017,1,6,PAH,Weight,DTXSID6063143,5-Methylchrysene,NA


,Sample,HAWC_ID,Study,Sample_Number,Number,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289737,Campbell et al. 2024,1,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
2,WF,1289737,Campbell et al. 2024,1,2,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,NA
3,WF,1289737,Campbell et al. 2024,1,3,PAH,Weight,DTXSID0029187,"2,6-Dimethylnaphthalene",NA
4,WF,1289737,Campbell et al. 2024,1,4,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,NA
5,WF,1289737,Campbell et al. 2024,1,5,PAH,Weight,DTXSID3061134,9-Methylanthracene,NA
6,WF,1289737,Campbell et al. 2024,1,6,PAH,Weight,DTXSID3023845,Acenaphthylene,NA


In [3]:
dim(ws_df)
dim(wf_df)

[1] 4391   10

[1] 3795   10

Starting with 6607 WS records and 4340 WF records. 

# Filter #1

Removing duplicate records for instances that weren't measured at all (MAR). 

In [4]:
`%notin%` <- Negate(`%in%`)

ws_df = ws_df %>%
    filter(Replicate %notin% NA)
wf_df = wf_df %>%
    filter(Replicate %notin% NA)

ERROR: [1m[33mError[39m in `filter()`:[22m
[1m[22m[36mℹ[39m In argument: `Replicate %notin% NA`.
[1mCaused by error:[22m
[33m![39m object 'Replicate' not found


In [6]:
# combining dfs
combined_df = rbind(ws_df, wf_df)
head(combined_df)

,Sample,HAWC_ID,Study,Replicate,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [7]:
length(unique(combined_df$Name))
length(unique(ws_df$Name))
length(unique(wf_df$Name))
#length(unique(control_df$Name))

[1] 192

[1] 137

[1] 138

228 unique metals/chemicals total.

# Filter #2: Chemical/Metal Background Filters

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. 

A variable (ie. `Name` which represents a chemical or metal) will be retained if >= 20% of its experimental data points were measurable values. In other words, chemicals or metals were excluded if they had NAs (not evaluated) or had non-detect values across >= 80% of the samples.

In [8]:
sample_type_presence_df = combined_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 20% experimental values for each chemical/metal
    group_by(Metric, Sample, Name) %>%
    # calculating the percentage of experimental data points
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Variable_Presence_Percentage >= 20) 

head(sample_type_presence_df)

Metric,Sample,Name,Variable_Presence_Percentage
<chr>,<chr>,<chr>,<dbl>
Volume,WF,"1,2,4-Trimethylbenzene",100
Volume,WF,"1,3,5-Trimethylbenzene",100
Volume,WF,"1,3-Dihydroxynaphthalene",100
Volume,WF,"1,4-Dichloro-2-butene, cis",100
Volume,WF,"1,4-Dichlorobenzene",100
Volume,WF,"1,8-Dihydroxynaphthalene",100


In [9]:
# filtering the original df
filter1_df = inner_join(sample_type_presence_df[,1:3], combined_df) %>%
    # metals (and some chemicals) don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(DTXSID != 'NA', DTXSID, Name)) 

head(filter1_df)

Joining with `by = join_by(Metric, Sample, Name)`


Metric,Sample,Name,HAWC_ID,Study,Replicate,Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,WF,"1,2,4-Trimethylbenzene",1289926,Wang et al. 2024,1,VOC,DTXSID6021402,730
Volume,WF,"1,2,4-Trimethylbenzene",1306371,Ketcherside et al. 2024,1,VOC,DTXSID6021402,130
Volume,WF,"1,3,5-Trimethylbenzene",1289926,Wang et al. 2024,1,VOC,DTXSID6026797,330
Volume,WF,"1,3,5-Trimethylbenzene",1306371,Ketcherside et al. 2024,1,VOC,DTXSID6026797,110
Volume,WF,"1,3-Dihydroxynaphthalene",1289821,Liang et al. 2021,1,PAH,DTXSID40456587,6
Volume,WF,"1,4-Dichloro-2-butene, cis",1289926,Wang et al. 2024,1,VOC,DTXSID3027405,230


In [10]:
dim(filter1_df)

[1] 4829    9

Started with 7796 records, 1786 were removed, leaving 6010. 

# Filter #3: Study Background Filter

A study will be retained if >= 20% of its experimental data points were measurable values. In other words, chemicals were excluded if they were NAs (not evaluated) or had non-detect values across < 20% of the samples.

In [11]:
study_type_presence_df = filter1_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 20% experimental values for each chemical/metal
    group_by(Metric, Class, HAWC_ID) %>%
    # calculating the percentage of experimental data points
    reframe(Study_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Study_Presence_Percentage >= 20) 

head(study_type_presence_df)

Metric,Class,HAWC_ID,Study_Presence_Percentage
<chr>,<chr>,<dbl>,<dbl>
Volume,Metal,266980,80.00000
Volume,Metal,266987,81.81818
Volume,Metal,452289,75.00000
Volume,Metal,885041,100.00000
Volume,Metal,993956,92.00000
Volume,Metal,1070627,100.00000


In [12]:
filter2_df = inner_join(study_type_presence_df[,1:3], filter1_df)

head(filter2_df)

Joining with `by = join_by(Metric, Class, HAWC_ID)`


Metric,Class,HAWC_ID,Sample,Name,Study,Replicate,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,1,K,9700
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,2,K,8800
Volume,Metal,266980,WS,Li,Stockfelt et al. 2012,1,Li,NA
Volume,Metal,266980,WS,Pb,Stockfelt et al. 2012,1,Pb,170
Volume,Metal,266980,WS,Pb,Stockfelt et al. 2012,2,Pb,400
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,1,Rb,73


In [13]:
dim(filter1_df)
dim(filter2_df)

[1] 4829    9

[1] 4804    9

299 more records were removed.

# Filter #4: Chemical/Metal Background Filter

In the intial filter, WS and WF samples were combined. However, this time variables (ie. metal or chemical) will be split based on their `Metric` (ie. volume or weight) and then retained if that variable is found in both woodsmoke and wildfire samples within volume or weight samples.

In [14]:
split_filtered_df = filter2_df %>%
    group_by(Sample, Metric) %>%
    group_split

split_ws_vol_df = split_filtered_df[[3]]
split_ws_weight_df = split_filtered_df[[4]]
split_wf_vol_df = split_filtered_df[[1]]
split_wf_weight_df = split_filtered_df[[2]]

In [15]:
# seeing how many unique variables are in each df and if they're consistent in each file
length(unique(split_ws_vol_df$Name))
length(unique(split_wf_vol_df$Name))
length(unique(split_ws_weight_df$Name))
length(unique(split_wf_weight_df$Name))

[1] 98

[1] 68

[1] 82

[1] 70

In [16]:
# they're not so first getting variables that are in weight or volume samples
consistent_wf_vol_df = split_wf_vol_df %>%
    filter(Name %in% unique(split_ws_vol_df$Name))
consistent_ws_vol_df = split_ws_vol_df %>%
    filter(Name %in% consistent_wf_vol_df$Name)
consistent_wf_weight_df = split_wf_weight_df %>%
    filter(Name %in% unique(split_ws_weight_df$Name))
consistent_ws_weight_df = split_ws_weight_df %>%
    filter(Name %in% consistent_wf_weight_df$Name)

length(unique(consistent_wf_vol_df$Name))
length(unique(consistent_ws_vol_df$Name))
length(unique(consistent_wf_weight_df$Name))
length(unique(consistent_ws_weight_df$Name))

[1] 33

[1] 33

[1] 31

[1] 31

There were 98, 68, 94 and 70 woodsmoke weight, woodsmoke volume, wildfire weight, and wildfire volume samples, respectively. 33 variables were common between volume samples and 34 were common between weight records and will be retained.

In [17]:
# recombining data
vol_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df)
weight_df = rbind(consistent_ws_weight_df, consistent_wf_weight_df)

head(vol_df)

Metric,Class,HAWC_ID,Sample,Name,Study,Replicate,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,1,K,9700
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,2,K,8800
Volume,Metal,266980,WS,Li,Stockfelt et al. 2012,1,Li,NA
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,1,Rb,73
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,2,Rb,105
Volume,Metal,266987,WS,Ca,Seagrave 2005,1,Ca,49200


# Square Root Imputation

Taking the square root of the min value to impute MNAR data.

In [18]:
# first determining number of ND and NA values in each df
vol_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

weight_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

Value,n()
<chr>,<int>
NA,5
ND,1


Value,n()
<chr>,<int>
NA,226
ND,74


In [19]:
dim(vol_df)
dim(weight_df)

[1] 750   9

[1] 1614    9

In [20]:
# creating a df for MAR data
mar_weight_df = weight_df %>%
    filter(Value == 'NA')
mar_vol_df = vol_df %>%
    filter(Value == 'NA')

preimputed_weight_df = anti_join(weight_df, mar_weight_df) 
preimputed_vol_df = anti_join(vol_df, mar_vol_df) 

preimputed_weight_df$Value = as.numeric(preimputed_weight_df$Value)
preimputed_vol_df$Value = as.numeric(preimputed_vol_df$Value)
mar_weight_df$Value = as.numeric(mar_weight_df$Value)
mar_vol_df$Value = as.numeric(mar_vol_df$Value)

# reordering cols
preimputed_weight_df = preimputed_weight_df[,c(3,1,4,6,7,2,8,5,9)]
preimputed_vol_df = preimputed_vol_df[,c(3,1,4,6,7,2,8,5,9)]

head(preimputed_weight_df)

Joining with `by = join_by(Metric, Class, HAWC_ID, Sample, Name, Study,
Replicate, DTXSID, Value)`
Joining with `by = join_by(Metric, Class, HAWC_ID, Sample, Name, Study,
Replicate, DTXSID, Value)`
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”


HAWC_ID,Metric,Sample,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cr,Cr,21.4
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cr,Cr,21.3
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cu,Cu,65.6
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cu,Cu,34.4
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Fe,Fe,1323.0
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Fe,Fe,1773.0


In [21]:
# creating a df for MNAR data
mnar_weight_df = preimputed_weight_df %>%
    filter(is.na(Value))
mnar_vol_df = preimputed_vol_df %>%
    filter(is.na(Value))

head(mnar_weight_df)

HAWC_ID,Metric,Sample,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Mn,Mn,NA
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Mn,Mn,NA
271638,Weight,WS,Jalava et al. 2012,1,Metal,Si,Si,NA
271638,Weight,WS,Jalava et al. 2012,2,Metal,Si,Si,NA
271638,Weight,WS,Jalava et al. 2012,3,Metal,Si,Si,NA
271638,Weight,WS,Jalava et al. 2012,5,Metal,Si,Si,NA


In [22]:
square_root_imputation = function(mnar_df, preimputed_df){
    # """
    # Imputing MNAR values based on the square root of the minimum concentration for each chemical 
    # within a study or metal.
    
    # :param: MNAR dataframe, preimputed dataframe
    # :output: a dataframe with imputed values

    # """

    id_df = mnar_df[is.na(mnar_df$Value),]
    study_ids = unique(id_df$HAWC_ID)
    
    imputed_df = data.frame()
    for (i in 1:length(id_df$HAWC_ID)){
    
        filtered_df = preimputed_df %>%
            filter(HAWC_ID == id_df$HAWC_ID[i], DTXSID == id_df$DTXSID[i])
    
        na_df = filtered_df %>%
            filter(is.na(Value)) %>%
            mutate(Value = sqrt(min(filtered_df$Value, na.rm = TRUE)))
    
        # combining rows of data
        combined_df = rbind(filtered_df %>%
            filter(!is.na(Value)), na_df)
    
        imputed_df = rbind(imputed_df, combined_df) %>%
                        unique()
    }
    return(imputed_df)    
    
}

# calling fn
imputed_vol_df = square_root_imputation(mnar_vol_df, preimputed_vol_df) %>%
    mutate(Value = ifelse(Value == Inf, NA, Value))
imputed_weight_df = square_root_imputation(mnar_weight_df, preimputed_weight_df) %>%
    mutate(Value = ifelse(Value == Inf, NA, Value))
    
head(imputed_weight_df)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; retur

HAWC_ID,Metric,Sample,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Mn,Mn,NA
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Mn,Mn,NA
271638,Weight,WS,Jalava et al. 2012,4,Metal,Si,Si,14700.0000
271638,Weight,WS,Jalava et al. 2012,1,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,2,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,3,Metal,Si,Si,121.2436


In [23]:
# there are still a few variables that didn't have any values to do the square root imputation
# their imputed values will be based off of distributions from other studies
other_studies_vol_df = semi_join(preimputed_vol_df %>%
                filter(!is.na(Value)), imputed_vol_df %>%
                               filter(is.na(Value)) %>%
    select(2,3,6,7)) %>%
    group_by(DTXSID) %>%
    summarize(Value = sqrt(min(Value)))

other_studies_weight_df = semi_join(preimputed_weight_df %>%
                filter(!is.na(Value)), imputed_weight_df %>%
                               filter(is.na(Value)) %>%
    select(2,3,6,7)) %>%
    group_by(DTXSID) %>%
    summarize(Value = sqrt(min(Value)))

head(other_studies_vol_df)
head(other_studies_weight_df)

Joining with `by = join_by(Metric, Sample, Class, DTXSID)`
Joining with `by = join_by(Metric, Sample, Class, DTXSID)`


DTXSID,Value
<chr>,<dbl>
Li,1.794436


DTXSID,Value
<chr>,<dbl>
Al,1.31148770
Ba,1.00000000
Co,0.10000000
Cr,0.31622777
DTXSID3021774,0.06033241
DTXSID5047740,0.04582576


In [24]:
# removing NAs from the imputed df 
removed_imputed_vol_df = imputed_vol_df %>%
    filter(!is.na(Value))
removed_imputed_weight_df = imputed_weight_df %>%
    filter(!is.na(Value))

# adding in the newly imputed values back in 
all_imputed_vol_df = left_join(imputed_vol_df %>%
        filter(is.na(Value)) %>%
        select(-Value), other_studies_vol_df)

all_imputed_weight_df = left_join(imputed_weight_df %>%
        filter(is.na(Value)) %>%
        select(-Value), other_studies_weight_df)

all_imputed_vol_df = rbind(removed_imputed_vol_df, all_imputed_vol_df)
all_imputed_weight_df = rbind(removed_imputed_weight_df, all_imputed_weight_df)

head(all_imputed_weight_df)

Joining with `by = join_by(DTXSID)`
Joining with `by = join_by(DTXSID)`


HAWC_ID,Metric,Sample,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
271638,Weight,WS,Jalava et al. 2012,4,Metal,Si,Si,14700.0000
271638,Weight,WS,Jalava et al. 2012,1,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,2,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,3,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,5,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,6,Metal,Si,Si,121.2436


In [25]:
# creating 1 df
mnar_imputed_vol_df = rbind(#imputed data
                               all_imputed_vol_df,
                               # adding in MAR data
                               mar_vol_df,
                               # adding back in non-imputed data
                               anti_join(preimputed_vol_df, all_imputed_vol_df[,1:8]))
mnar_imputed_weight_df = rbind(# imputed data
                               all_imputed_weight_df,
                               # adding in MAR data
                               mar_weight_df,
                               # adding back in non-imputed data
                               anti_join(preimputed_weight_df, all_imputed_weight_df[,1:8]))

head(mnar_imputed_weight_df)
dim(mnar_imputed_vol_df)
dim(mnar_imputed_weight_df)

Joining with `by = join_by(HAWC_ID, Metric, Sample, Study, Replicate, Class,
DTXSID, Name)`
Joining with `by = join_by(HAWC_ID, Metric, Sample, Study, Replicate, Class,
DTXSID, Name)`


HAWC_ID,Metric,Sample,Study,Replicate,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
271638,Weight,WS,Jalava et al. 2012,4,Metal,Si,Si,14700.0000
271638,Weight,WS,Jalava et al. 2012,1,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,2,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,3,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,5,Metal,Si,Si,121.2436
271638,Weight,WS,Jalava et al. 2012,6,Metal,Si,Si,121.2436


[1] 750   9

[1] 1614    9

# Filter #4: Double Checking MAR Records

Making sure that when the data is split based on `Class` and transformed into a wide format, we have at least 20% of the records with numeric values for each metal or chemical.

In [26]:
avg_vol_df = mnar_imputed_vol_df %>%
    # averaging to get rid of replicates
    group_by(HAWC_ID, DTXSID) %>%
    mutate(Avg_Value = mean(as.numeric(Value))) %>%
    ungroup() %>%
    select(-c("Replicate", "Value", "Name", "Class")) %>%
    unique() 

head(avg_vol_df)

HAWC_ID,Metric,Sample,Study,DTXSID,Avg_Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
993956,Volume,WS,Ihantola et al. 2022,Li,1.794436
266980,Volume,WS,Stockfelt et al. 2012,Li,NA
266987,Volume,WS,Seagrave 2005,Li,NA
452289,Volume,WS,Rabha 2018,Li,NA
1263480,Volume,WS,Burnet et al. 1990,Li,NA
1263484,Volume,WS,Leese et al. 1989,Li,NA


In [27]:
avg_weight_df = mnar_imputed_weight_df %>%
    # averaging to get rid of replicates
    group_by(HAWC_ID, DTXSID) %>%
    mutate(Avg_Value = mean(Value, na.rm = TRUE)) %>%
    ungroup() %>%
    select(-c("Replicate", "Value", "Name", "Class")) %>%
    unique() 

head(avg_weight_df)

HAWC_ID,Metric,Sample,Study,DTXSID,Avg_Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
271638,Weight,WS,Jalava et al. 2012,Si,2203.92305
299223,Weight,WS,Kasurinen et al. 2015,Ca,17284.47712
299223,Weight,WS,Kasurinen et al. 2015,Ni,11.05409
1040882,Weight,WS,Nordin et al. 2015,Ca,3602.19005
1040882,Weight,WS,Nordin et al. 2015,Fe,603.11724
1063054,Weight,WS,Corsini et al. 2013,Cr,23.16228


Converting values to 0s and 1s to count number of experimental and NDs previously imputed. Chemicals/metals that don't have at least 50% of values present will be removed.

In [28]:
wider_vol_df = avg_vol_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(avg_vol_df)[c(1,3)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value") 

wider_weight_df = avg_weight_df %>%
    unite("SampleID", c(colnames(avg_weight_df)[c(1,3)]), remove = FALSE) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value")   

head(wider_weight_df)

SampleID,HAWC_ID,Metric,Sample,Study,Si,Ca,Ni,Fe,Cr,⋯,Mg,B,DTXSID4020878,DTXSID0047741,DTXSID4075455,DTXSID4075459,DTXSID2060383,DTXSID8074819,DTXSID4020161,DTXSID1025649
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
271638_WS,271638,Weight,WS,Jalava et al. 2012,2203.923048,2102.000,NaN,NaN,NaN,⋯,341.4285714,NA,NA,NA,NA,235.4286,NA,NA,NA,NA
299223_WS,299223,Weight,WS,Kasurinen et al. 2015,NaN,17284.477,11.0540926,NaN,NaN,⋯,NA,NA,NA,NA,NA,0.4500,NA,NA,NA,NA
1040882_WS,1040882,Weight,WS,Nordin et al. 2015,NaN,3602.190,0.1732051,603.1172428,129.0000000,⋯,1791.0000000,288,NA,NA,NA,NA,NA,NA,NA,NA
1063054_WS,1063054,Weight,WS,Corsini et al. 2013,5675.000000,4385.000,30.0000000,355.0000000,23.1622777,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1098462_WS,1098462,Weight,WS,Niu et al. 2023,3.226667,1.010,0.0400000,0.9933333,0.1466667,⋯,0.9533333,NA,NA,NA,NA,NA,NA,NA,NA,NA
271769_WS,271769,Weight,WS,Uski et al. 2012,NaN,2130.571,NaN,NaN,NaN,⋯,341.4285714,NA,NA,NA,NA,234.4857,NA,NA,NA,NA


In [29]:
data_percentage = function(wider_dataset){
    # """
    # Calculating the studies or metals/chemicals with less than or equal to 20% of experimental data.

    # :param: wide dataframe
    # :output: a list that contains a dataframe of metals/chemicals and a dataframe of studies

    # """
    
    # converting values to 0s and 1s
    summed_df = wider_dataset %>%
        mutate(across(7:ncol(wider_dataset), ~ifelse(is.na(.x), 0, 1)))

    # identifying metals/chemicals w/ =< 20% of experimental data
    var_percentage_df = data.frame(Percent = colSums(summed_df[,7:ncol(summed_df)])/nrow(summed_df) * 100) %>%
        filter(Percent <= 20)
    # identifying studies w/ =< 20% of experimental data
    study_percentage_df = data.frame(HAWC_ID = wider_dataset$HAWC_ID,
                                     Percent = rowSums(summed_df[,7:ncol(summed_df)])/ncol(summed_df) * 100) %>%
        filter(Percent <= 20)

    return(list(var_percentage_df, study_percentage_df))
}

# calling fn
percentage_vol_df = data_percentage(wider_vol_df)
percentage_weight_df = data_percentage(wider_weight_df)

head(percentage_weight_df[[1]])

,Percent
,<dbl>
Rb,12.280702
DTXSID5047740,15.789474
DTXSID8052691,12.280702
DTXSID6062591,14.035088
DTXSID0025654,7.017544
DTXSID9059757,14.035088


In [30]:
# removing those metals/chemicals & studies from the OG df
preimputed_vol_df = wider_vol_df[,colnames(wider_vol_df) %notin% rownames(percentage_vol_df[[1]])] %>%
    select(-DTXSID8020913)
preimputed_weight_df = wider_weight_df[,colnames(wider_weight_df) %notin% rownames(percentage_weight_df[[1]])]

preimputed_vol_df = preimputed_vol_df %>% filter(HAWC_ID %notin% percentage_vol_df[[2]]$HAWC_ID) 
preimputed_weight_df = preimputed_weight_df %>% filter(HAWC_ID %notin% percentage_weight_df[[2]]$HAWC_ID)

head(preimputed_weight_df)

SampleID,HAWC_ID,Metric,Sample,Study,Si,Ca,Ni,Fe,Cr,⋯,DTXSID6024254,DTXSID3024104,Mn,Al,Co,Ti,Cu,Zn,K,Mg
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
271638_WS,271638,Weight,WS,Jalava et al. 2012,2203.923048,2102.000,NaN,NaN,NaN,⋯,3.590800e+03,2137.0857143,422.2857143,NA,NaN,NaN,NA,5202.857143,141557.1429,341.4285714
299223_WS,299223,Weight,WS,Kasurinen et al. 2015,NaN,17284.477,11.0540926,NaN,NaN,⋯,1.166667e+00,64.5666667,NaN,NaN,NaN,NaN,NA,6503.333333,262000.0000,NA
1040882_WS,1040882,Weight,WS,Nordin et al. 2015,NaN,3602.190,0.1732051,603.1172428,129.0000000,⋯,NA,NA,0.4582576,638.00,0.100,0.17320508,266.00,4660.000000,22500.0000,1791.0000000
1063054_WS,1063054,Weight,WS,Corsini et al. 2013,5675.000000,4385.000,30.0000000,355.0000000,23.1622777,⋯,NA,NA,945.0000000,685.00,NaN,0.17320508,375.00,10380.000000,NA,NA
1098462_WS,1098462,Weight,WS,Niu et al. 2023,3.226667,1.010,0.0400000,0.9933333,0.1466667,⋯,6.666667e-02,0.2714214,0.2700000,2.61,0.015,0.03666667,0.35,2.076667,168.3033,0.9533333
271769_WS,271769,Weight,WS,Uski et al. 2012,NaN,2130.571,NaN,NaN,NaN,⋯,3.642000e+03,2137.5714286,422.2857143,NaN,NaN,NaN,NA,5203.142857,141557.1429,341.4285714


In [31]:
# How many metals/chemicals are now left in the dataset?
ncol(preimputed_vol_df) - 6
ncol(preimputed_weight_df) - 6

[1] 20

[1] 15

In [32]:
# How many studies are left?
nrow(preimputed_vol_df)  
nrow(preimputed_weight_df) 

[1] 11

[1] 31

ADD WORDS. 

# RF Imputation

In [33]:
random_forest_imputation = function(dataset){
    # """
    # Using random forest (RF) to generate MAR data.

    # :param: wide dataframe
    # :output: a longer dataframe 

    # """
    
    preimputed_df = dataset[,6:ncol(dataset)]
    
    imputed_RF_object = missForest(as.matrix(preimputed_df))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:5], imputed_RF_df)) %>%
        pivot_longer(cols = 6:ncol(dataset), names_to = "DTXSID", values_to = "Value")
    
    return(imputed_final_df)
}

# calling fn
rf_vol_imputed_df = random_forest_imputation(preimputed_vol_df) 
rf_weight_imputed_df = random_forest_imputation(preimputed_weight_df)

rf_imputed_df = rbind(rf_vol_imputed_df, 
                      rf_weight_imputed_df) 

head(rf_imputed_df)
dim(rf_imputed_df)

Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response h

SampleID,HAWC_ID,Metric,Sample,Study,DTXSID,Value
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
993956_WS,993956,Volume,WS,Ihantola et al. 2022,Li,1.794436
993956_WS,993956,Volume,WS,Ihantola et al. 2022,K,60000.000000
993956_WS,993956,Volume,WS,Ihantola et al. 2022,Ca,2800.000000
993956_WS,993956,Volume,WS,Ihantola et al. 2022,Cd,25.000000
993956_WS,993956,Volume,WS,Ihantola et al. 2022,Fe,1400.000000
993956_WS,993956,Volume,WS,Ihantola et al. 2022,Mn,280.000000


[1] 727   7

In [34]:
# adding back in name col
final_df = left_join(rf_imputed_df, 
                      combined_df[,c(5,7:8)] %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(grepl("NA", DTXSID), Name, DTXSID)) %>%
    mutate(Name = ifelse(Name == 'Dibenz(ah)anthracene', 'Dibenzo(ah)anthracene', 
                    ifelse(Name == 'Benz(a)anthracene', 'Benzo(a)anthracene', 
                        ifelse(Name == 'o-Xylene', 'O-xylene', 
                            ifelse(Name == 'Propylbenzene (Benzene, propyl)', 'Propylbenzene', 
                                ifelse(Name == 'n-Hexane', 'Hexane', 
                                    ifelse(Name == 'Carbon tetrachloride (tetrachloromethane)', 'Carbon tetrachloride',
                                        ifelse(Name == 'Trichloromethane (Chloroform)', 'Chloroform', Name)))))))) %>%
    unique()) 

final_df = final_df[,c(1,2,5,3,4,8,6,9,7)]

head(final_df)
dim(final_df)

Joining with `by = join_by(DTXSID)`


SampleID,HAWC_ID,Study,Metric,Sample,Class,DTXSID,Name,Value
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
993956_WS,993956,Ihantola et al. 2022,Volume,WS,Metal,Li,Li,1.794436
993956_WS,993956,Ihantola et al. 2022,Volume,WS,Metal,K,K,60000.000000
993956_WS,993956,Ihantola et al. 2022,Volume,WS,Metal,Ca,Ca,2800.000000
993956_WS,993956,Ihantola et al. 2022,Volume,WS,Metal,Cd,Cd,25.000000
993956_WS,993956,Ihantola et al. 2022,Volume,WS,Metal,Fe,Fe,1400.000000
993956_WS,993956,Ihantola et al. 2022,Volume,WS,Metal,Mn,Mn,280.000000


[1] 727   9

In [35]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "Processed_Data_", cur_date, ".xlsx"), 
           rowNames = FALSE)